In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fellow-6/Train.csv
/kaggle/input/fellow-6/Test.csv


## Functions

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Read Data

In [ ]:
train_data = pd.read_csv("/kaggle/input/fellow-6/Train.csv")
train_data.head()

,date,day,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Thursday,0.65,23.69,861.0,7200,0,0.0,0,0,60.0,0.521180
1,1/3/2015,Saturday,0.70,23.69,544.0,13800,0,0.0,0,0,60.0,0.700134
2,1/4/2015,Sunday,0.80,28.08,878.0,4260,50,0.0,0,0,55.5,0.800598
3,1/5/2015,Monday,0.80,28.08,759.0,6900,50,0.0,0,0,57.5,0.800077
4,1/6/2015,Tuesday,0.80,28.08,925.0,10350,50,0.0,0,0,57.5,0.800594


In [ ]:
test_data = pd.read_csv("/kaggle/input/fellow-6/Test.csv")
test_data.head()

,date,day,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers
0,2/26/2015,Thursday,0.80,30.10,437.0,7080,32,0.0,0,2,59.0
1,2/28/2015,Saturday,0.50,26.66,556.0,6960,23,0.0,0,2,58.0
2,3/1/2015,Sunday,0.60,26.66,692.0,6840,23,0.0,0,0,57.0
3,3/2/2015,Monday,0.65,26.66,832.0,6780,55,0.0,0,0,57.0
4,3/3/2015,Tuesday,0.70,26.66,950.0,6960,50,0.0,0,0,58.0


## Check Dataset

In [ ]:
print(train_data.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(48, 12)
Series([], dtype: int64)


In [ ]:
print(test_data.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(11, 11)
Series([], dtype: int64)


In [ ]:
train_d = train_data.drop(['date'], axis=1)

In [ ]:
# Get list of categorical variables
s = (train_d.dtypes == 'object')
train_object_cols = list(s[s].index)

print("Categorical variables:")
print(train_object_cols)

Categorical variables:
['day']


In [ ]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train_d[train_object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = train_d.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = train_d.drop(train_object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)

In [ ]:
OH_X_train.head()

,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,0,1,2,3,4,5
0,0.65,23.69,861.0,7200,0,0.0,0,0,60.0,0.521180,0.0,0.0,0.0,1.0,0.0,0.0
1,0.70,23.69,544.0,13800,0,0.0,0,0,60.0,0.700134,0.0,1.0,0.0,0.0,0.0,0.0
2,0.80,28.08,878.0,4260,50,0.0,0,0,55.5,0.800598,0.0,0.0,1.0,0.0,0.0,0.0
3,0.80,28.08,759.0,6900,50,0.0,0,0,57.5,0.800077,1.0,0.0,0.0,0.0,0.0,0.0
4,0.80,28.08,925.0,10350,50,0.0,0,0,57.5,0.800594,0.0,0.0,0.0,0.0,1.0,0.0


## Slipt  Dataset

In [ ]:
y = OH_X_train.actual_productivity
# Create X 
features= OH_X_train.drop('actual_productivity', axis=1)

X = features

# Split into validation and training data
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

## Training

In [ ]:
forest_model = RandomForestRegressor(random_state=42)
forest_model.fit(X_train, y_train)
preds = forest_model.predict(X_val)
print(mean_absolute_percentage_error(y_val, preds))

7.132679894532597


In [ ]:
forest_model1 = RandomForestRegressor(n_estimators=50, random_state=42)
forest_model1.fit(X_train, y_train)
preds1 = forest_model1.predict(X_val)
print(mean_absolute_percentage_error(y_val, preds1))

7.498137362702377


In [ ]:
model = RandomForestRegressor(max_depth=3, min_samples_split=6,random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_val)
print(mean_absolute_percentage_error(y_val, pred))

7.606747397187262


In [ ]:
modell = RandomForestRegressor(max_depth=5, min_samples_split=3,random_state=42)
modell.fit(X_train, y_train)
pre = modell.predict(X_val)
print(mean_absolute_percentage_error(y_val, pre))

7.047907146854637


In [ ]:
model1 = RandomForestRegressor(max_depth=5, min_samples_split=3)
model1.fit(X_train, y_train)
pred1 = model1.predict(X_val)
print(mean_absolute_percentage_error(y_val, pred1))

7.371729347462446


## Test Data

In [ ]:
test_d = test_data.drop(['date'], axis=1)

In [ ]:
# Get list of categorical variables
s = (test_d.dtypes == 'object')
test_object_cols = list(s[s].index)

print("Categorical variables:")
print(test_object_cols)

Categorical variables:
['day']


In [ ]:
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_test = pd.DataFrame(OH_encoder.fit_transform(test_d[test_object_cols]))

# One-hot encoding removed index; put it back
OH_cols_test.index = test_d.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_test = test_d.drop(test_object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

In [ ]:
OH_X_test.head()

,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,0,1,2,3,4,5
0,0.80,30.10,437.0,7080,32,0.0,0,2,59.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.50,26.66,556.0,6960,23,0.0,0,2,58.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.60,26.66,692.0,6840,23,0.0,0,0,57.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.65,26.66,832.0,6780,55,0.0,0,0,57.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.70,26.66,950.0,6960,50,0.0,0,0,58.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Fill in the line below: get test predictions
preds_test = model1.predict(OH_X_test)

## Save submission

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'id': test_data.index,
                       'actual_productivity': preds_test})
output.to_csv('submission.csv', index=False)